In [1]:
import numpy as np
def calculate_angle(a,b,c): #calculate angle given 3 connected joints
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0]) #angle from endpoint to midpoint subtracted wiht angle from midpoint to firstpoint 
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [7]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached mediapipe-0.8.11-cp310-cp310-win_amd64.whl (49.0 MB)
  Using cached matplotlib-3.6.1-cp310-cp310-win_amd64.whl (7.2 MB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached attrs-22.1.0-py2.py3-none-any.whl (58 kB)
  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)
  Using cached kiwisolver-1.4.4-cp310-cp310-win_amd64.whl (55 kB)
  Using cached contourpy-1.0.5-cp310-cp310-win_amd64.whl (164 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.37.4-py3-none-any.whl (960 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Kridbhume Chammanard\\AppData\\Roaming\\Python\\Python310\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils #used for visualising our drawings
mp_pose = mp.solutions.pose #import pose estimation models

cap = cv2.VideoCapture(0)

#Instantiate mediapipe
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
#FEED
    while True:
        ret,frame = cap.read()

        #recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #saves memory

        #make detection
        results = pose.process(image) #store detections in results

        #recolor image back for cv2 to detect
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            hand_raised = False

            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_r = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_r  = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

            shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_l = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_l = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            hip_l = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee_l = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

            if (elbow_r[1] <= shoulder_r[1]):
                cv2.putText(image,"HAND RAISED", tuple(np.multiply(elbow_r,[640,480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)
                print("HAND RAISED")
                hand_raised = True

            elif (elbow_l[1] <= shoulder_l[1]):
                cv2.putText(image,"HAND RAISED", tuple(np.multiply(elbow_l,[640,480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)
                print("HAND RAISED")  
                hand_raised = True
            

            
            if (knee_r <= hip_r):
                cv2.putText(image,"SITTING", tuple(np.multiply(hip_r,[640,480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)
            elif (knee_l <= hip_l):
                cv2.putText(image,"SITTING", tuple(np.multiply(hip_l,[640,480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)

            

            cv2.putText(image, str(elbow_r[0]), tuple(np.multiply(elbow_r,[640,480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)

            
        except:
            pass
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color = (245,117,66), thickness = 2, circle_radius = 2),
        mp_drawing.DrawingSpec(color = (245,66,230), thickness = 2, circle_radius = 2)) 

        cv2.imshow("Feed",image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
